In [2]:
import pandas as pd
import numpy as np
# Stop pandas from truncating output.
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [36]:
net_runtime_key = "net_runtime_s"
tables_dir = "../../write_up/thesis/tables/"

def aggregate_readings(df: pd.DataFrame):
    df = df[["name", "threads", "delay", net_runtime_key]]
    df = df.groupby(
        by=["name", "threads", "delay"], 
        as_index=False
    ).agg(mean=(net_runtime_key, np.mean), std=(net_runtime_key, np.std))
    df["cov(%)"] = ((df["std"]*100)/df["mean"]).round(3)
    return df

df_pairwise = aggregate_readings(pd.read_csv("../../enqueue_dequeue_results.csv"))
df_coin_toss = aggregate_readings(pd.read_csv("../../p_enqueue_dequeue_results.csv"))

queue_names = ["MS Queue", "MS Two", "Baskets Queue", "Valois Queue"]
dataframes = [("Pairwise df", df_pairwise), ("50% Enqueue", df_coin_toss)]

def filter_readings(df: pd.DataFrame, queue_name: str, threads: int):
    return df.loc[
        (df["name"].str.contains(queue_name)) & (df["threads"] == threads )
    ].sort_values(by="mean")


# Performance Difference Between One and Two Threads

In [40]:
def get_difference_in_time_between_threads(df: pd.DataFrame, table_title: str):
    filtered_df = df.loc[df["threads"] <= 2][
        ["name", "threads", "delay", "mean"]
    ]
    
    filtered_df = filtered_df.groupby(
        by=["name", "delay"],
        as_index=False
    )
    
    filtered_df = filtered_df["mean"].apply(lambda x: x.iloc[1]/x.iloc[0])
    
    filtered_df = filtered_df.groupby(
        by=["name"],
        as_index=False
    ).agg(mean=("mean", "mean"), std=("mean", "std"))

    filtered_df["mean"] = filtered_df["mean"].round(3)
    filtered_df["std"] = filtered_df["std"].round(3)

    
    filtered_df = filtered_df.rename(
        columns={"name":"Queue", "mean":"Mean", "std":"$\sigma$"}
    )
    
    filtered_df.style.set_caption(table_title)
    return filtered_df.sort_values(by="Queue")

# Taken from: https://stackoverflow.com/questions/71895146/pandas-to-latex-how-to-make-column-names-bold
def save_differences_to_latex_table(filtered_df: pd.DataFrame, file_name: str, caption:str, label:str):
    (filtered_df
        .style
        .hide(axis="index")
        .applymap_index(lambda _: "font-weight:bold;", axis="columns")
        .to_latex(
            file_name,
            convert_css=True,
            environment="table*",
            position_float="centering",
            caption=caption,
            label=label,
            position="h"
        )
    )

In [41]:
pairwise_perf_deg = get_difference_in_time_between_threads(df_pairwise, "Pairwise Benchmark")
save_differences_to_latex_table(
    pairwise_perf_deg,
    f"{tables_dir}pairwise_1_to_2_thread_perfdeg.tex",
    "Magnitude of degradation in performance observed between one and two threads on the pairwise benchmark.",
    "tab:pairwise_1_to_2_perfdeg"
)
pairwise_perf_deg.head(100)

,Queue,Mean,$\sigma$
0,Baskets Queue using DWCAS,2.347,0.258
1,Baskets Queue using Tagged Pointers,4.269,0.584
2,MS Queue using DWCAS,2.489,0.195
3,MS Queue using Tagged Pointers,4.535,0.889
4,MS Two-Lock TTAS,4.166,0.863
5,Valois Queue using DWCAS,2.109,0.073
6,Valois Queue using Tagged Pointers,4.351,0.844


In [42]:
coin_toss_perf_deg = get_difference_in_time_between_threads(df_coin_toss, "50% Enqueue Benchmark")
save_differences_to_latex_table(
    coin_toss_perf_deg, 
    f"{tables_dir}coin_toss_1_to_2_thread_perfdeg.tex",
    "Magnitude of degradation in performance observed between one and two threads on the 50\% enqueue benchmark.",
    "tab:coin_toss_1_to_2_perfdeg"
)
coin_toss_perf_deg.head(100)

,Queue,Mean,$\sigma$
0,Baskets Queue using DWCAS,2.235,0.451
1,Baskets Queue using Tagged Pointers,2.865,0.333
2,MS Queue using DWCAS,2.371,0.479
3,MS Queue using Tagged Pointers,3.051,0.574
4,MS Two-Lock TTAS,3.087,0.599
5,Valois Queue using DWCAS,2.089,0.280
6,Valois Queue using Tagged Pointers,3.194,0.434


In [3]:
filter_readings(df_pairwise, "MS Queue", 1)

,name,threads,delay,mean,std,cov(%)
330,MS Queue using Tagged Pointers,1,500,0.320153,0.001428,0.446
325,MS Queue using Tagged Pointers,1,50,0.476307,0.001992,0.418
327,MS Queue using Tagged Pointers,1,150,0.481545,0.001919,0.398
326,MS Queue using Tagged Pointers,1,100,0.482798,0.000522,0.108
329,MS Queue using Tagged Pointers,1,250,0.497177,0.000402,0.081
328,MS Queue using Tagged Pointers,1,200,0.512181,0.001318,0.257
332,MS Queue using Tagged Pointers,1,1000,0.579488,0.001307,0.225
331,MS Queue using Tagged Pointers,1,750,0.597312,0.000623,0.104
324,MS Queue using Tagged Pointers,1,0,0.885010,0.000459,0.052
222,MS Queue using DWCAS,1,500,1.334365,0.011341,0.850


In [4]:
filter_readings(df_coin_toss, "MS Queue", 1)

,name,threads,delay,mean,std,cov(%)
326,MS Queue using Tagged Pointers,1,100,0.295568,0.000509,0.172
327,MS Queue using Tagged Pointers,1,150,0.298891,0.000771,0.258
325,MS Queue using Tagged Pointers,1,50,0.305157,0.000127,0.042
330,MS Queue using Tagged Pointers,1,500,0.329215,0.024443,7.425
329,MS Queue using Tagged Pointers,1,250,0.363548,0.002749,0.756
328,MS Queue using Tagged Pointers,1,200,0.365440,0.001573,0.430
332,MS Queue using Tagged Pointers,1,1000,0.383925,0.001164,0.303
331,MS Queue using Tagged Pointers,1,750,0.398807,0.001778,0.446
324,MS Queue using Tagged Pointers,1,0,0.511877,0.000072,0.014
222,MS Queue using DWCAS,1,500,0.694906,0.003261,0.469


In [5]:
df_ms_queue_eq_2_t = df_ms_queue_lt_4_t.loc[df_ms_queue_lt_4_t["threads"] == 2].sort_values(by="mean")
df_ms_queue_eq_2_t.head(100)

NameError: name 'df_ms_queue_lt_4_t' is not defined

In [ ]:
df_ms_queue_eq_3_t = df_ms_queue_lt_4_t.loc[df_ms_queue_lt_4_t["threads"] == 3].sort_values(by="mean")
df_ms_queue_eq_3_t.head(100)

In [ ]:
df_ms_queue_eq_4_t = df_ms_queue_lt_4_t.loc[df_ms_queue_lt_4_t["threads"] == 4].sort_values(by="mean")
df_ms_queue_eq_4_t.head(100)